# Xiaolun Gong LAB1

## Part 1

In [2]:
#import arcpy
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')
import arcpy
#set up feature class
file_name = "C:/Users/andygong/Downloads/saep_bg10/saep_bg10.shp"
table_name =  "C:/Users/andygong/Downloads/WashingtonFIPS.dbf"

Import tool to convert shp file to geoJSON files

In [3]:
# set up the environment variables for GDAL/OGR
from subprocess import call
import os
os.environ["GDAL_DATA"] = "C:\\OSGeo4W\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W\\apps\\msys\\bin;C:\\OSGeo4W\\apps\\Python27\\Scripts"

In [4]:
#set up workspace
arcpy.env.workspace = "C:/Users/andygong/Downloads/WA_Counties"
arcpy.env.overwriteOutput = True

In [5]:
# Create new field for county names
arcpy.AddField_management(file_name, "cntyName", "TEXT")

# get county name by loop
my_List = []
cursur = arcpy.da.SearchCursor(table_name, ["CountyName", "FIPSCounty"])
for row in cursur:
    my_List.append(row[0])
    # Update county name for new field in the shp
    cursur1 = arcpy.da.UpdateCursor(file_name, "cntyName", '"COUNTYFP10" = ' + "'%s'" %row[1])
    for row1 in cursur1:
        row1[0] = row[0]
        cursur1.updateRow(row1)
del cursur
del cursur1

In [10]:
# Export shapefiles for each county
for name in my_List:
    arcpy.FeatureClassToFeatureClass_conversion(file_name, "C:/Users/andygong/Downloads/WA_Counties/",
                                                "" + name, '"cntyName" = ' + "'%s'" %name)

check the spatial reference list use the project 2927

In [14]:

# Convert all .shp files into geoJSON files
for name in my_List:
    call(['C:\\OSGeo4W\\bin\\ogr2ogr', 
          '-f','GeoJSON','-t_srs','WGS84', 
          '-s_srs','EPSG:2927', 
         'C:/Users/andygong/Downloads/WA_Counties_GeoJSON/' + name + '.geojson', 
        'C:/Users/andygong/Downloads/WA_Counties/' + name + '.shp'])

WindowsError: [Error 2] The system cannot find the file specified

## Part 2

first of all, I created a list to list each counties' population.Second, I list them to descending order, then print out the first 10. 

In [12]:
# Add total population from each county to a list
popList = []
for name in my_List:
    pop = 0
    cursur = arcpy.da.SearchCursor(file_name, ["POP2017"], '"cntyName" = ' + "'%s'" %name)
                                
    for row in cursur:
        pop += row[0]
    popList.append(pop)
    
# make it to descending order
popList.sort(reverse=True)

# Print out population in descending order
i = 0
while i < 10:
    print popList[i]
    i += 1

2153700.0
859400.0
789400.0
499800.0
471000.0
276900.0
264300.0
253000.0
216300.0
193500.0


### REFLECTION

 Honestyly, this lab is really hard for me, especially the first part of the lab. I went through the Chapter 7 several times to see what I can do. It really helps. Also, thanks for professor, the post on disscussion board on Wednesday was really helpful. I have worked on this lab for over 12 hours. I did learn a lot from this lab. I've learnt the cursur from the textbook, learnt the whereclause from Kim. However, I am still having problems to address the name from shp files to the geoJSON files. Since it reaches the due time, i will work on weekends to see I can finish it.